<a href="https://colab.research.google.com/github/Chubbyman2/Clarisse/blob/master/Pytorch_Clarisse_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install nltk

In [21]:
import nltk
nltk.download("punkt") # Package with a pre-trained tokenizer
from nltk.stem.porter import PorterStemmer # Import stemmer
stemmer = PorterStemmer()
import numpy as np

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


**Helper Functions**

In [8]:
def tokenize(sentence):
  return nltk.word_tokenize(sentence)

In [7]:
def stem(word):
  return stemmer.stem(word.lower())

In [31]:
def bag_of_words(tokenized_sentence, all_words):
  '''
  sentence = ["hello", "how", "are", "you"]
  words = ["hi", "hello", "I", "you", "bye", "thank", "cool"]
  bag =   [ 0 ,    1   ,  0 ,   1  ,   0  ,    0   ,    0  ]
  '''
  tokenized_sentence = [stem(w) for w in tokenized_sentence]

  bag = np.zeros(len(all_words), dtype=np.float32)
  for idx, w in enumerate(all_words):
    if w in tokenized_sentence:
      bag[idx] = 1.0
    
  return bag

In [9]:
# Tokenization test
test = "How are you?"
print(tokenize(test))

['How', 'are', 'you', '?']


In [10]:
# Stemming test
test_words = ["Universal", "university", "universe"]
stemmed_words = [stem(w) for w in test_words]
print(stemmed_words)

['univers', 'univers', 'univers']


In [32]:
# Bag of words test
sentence = ["hello", "how", "are", "you"]
words = ["hi", "hello", "I", "you", "bye", "thank", "cool"]
bag = bag_of_words(sentence, words)
print(bag)

[0. 1. 0. 1. 0. 0. 0.]


**Data Preparation**

In [11]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [14]:
import json

with open("/content/gdrive/My Drive/intents.json", "r") as file:
  intents = json.load(file)

print(intents)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'Hey', 'How are you', 'Is anyone there?', 'Hello', 'Good day'], 'responses': ['Hey :-)', 'Hello, thanks for visiting', 'Hi there, what can I do for you?', 'Hi there, how can I help?']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye'], 'responses': ['See you later, thanks for visiting', 'Have a nice day', 'Bye! Come back again soon.']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", "Thank's a lot!"], 'responses': ['Happy to help!', 'Any time!', 'My pleasure']}, {'tag': 'items', 'patterns': ['Which items do you have?', 'What kinds of items are there?', 'What do you sell?'], 'responses': ['We sell coffee and tea', 'We have coffee and tea']}, {'tag': 'payments', 'patterns': ['Do you take credit cards?', 'Do you accept Mastercard?', 'Can I pay with Paypal?', 'Are you cash only?'], 'responses': ['We accept VISA, Mastercard and Paypal', 'We accept most major credit cards, and Paypal']}, {'tag': 'd

In [17]:
all_words = []
tags = []
word_and_tag = []

for intent in intents["intents"]:
  tag = intent["tag"]
  tags.append(tag)
  for pattern in intent["patterns"]:
    w = tokenize(pattern)
    all_words.extend(w)
    word_and_tag.append((w, tag))

In [20]:
ignore_words = ["?", "!", ".", ","]
all_words = [stem(w) for w in all_words if w not in ignore_words] # Stems all the words
all_words = sorted(set(all_words)) # All unique words, sorted in order
print(all_words) # Prints unique, stemmed words

tags = sorted(set(tags))
print(tags)

["'s", 'a', 'accept', 'anyon', 'are', 'bye', 'can', 'card', 'cash', 'credit', 'day', 'deliveri', 'do', 'doe', 'funni', 'get', 'good', 'goodbi', 'have', 'hello', 'help', 'hey', 'hi', 'how', 'i', 'is', 'item', 'joke', 'kind', 'know', 'later', 'long', 'lot', 'mastercard', 'me', 'my', 'of', 'onli', 'pay', 'paypal', 'see', 'sell', 'ship', 'someth', 'take', 'tell', 'thank', 'that', 'there', 'what', 'when', 'which', 'with', 'you']
['delivery', 'funny', 'goodbye', 'greeting', 'items', 'payments', 'thanks']


In [38]:
x_train = [] 
y_train = []

for (pattern_sentence, tag) in word_and_tag:
  bag = bag_of_words(pattern_sentence, all_words)
  x_train.append(bag)

  label = tags.index(tag)
  y_train.append(label)

x_train = np.array(x_train)
y_train = np.array(y_train)

**Pytorch Model**

In [39]:
import torch
import torch.nn as nn 
from torch.utils.data import Dataset, DataLoader

In [67]:
class ChatDataset(Dataset):
  def __init__(self):
    self.n_samples = len(x_train)
    self.x_data = x_train
    self.y_data = y_train

  def __getitem__(self, index):
    return self.x_data[index], self.y_data[index]
  
  def __len__(self):
    return self.n_samples

In [83]:
# Set hyperparameters
batch_size = 8
hidden_size = 8
input_size = len(all_words)
output_size = len(tags)
learning_rate = 0.001
num_epochs = 1000

dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=0)

print(input_size, output_size)

54 54 7


In [94]:
class NeuralNet(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(NeuralNet, self).__init__()
    self.l1 = nn.Linear(input_size, hidden_size)
    self.l2 = nn.Linear(hidden_size, hidden_size)
    self.l3 = nn.Linear(hidden_size, num_classes)
    self.relu = nn.ReLU()

  def forward(self, x):
    out = self.l1(x)
    out = self.relu(out)
    out = self.l2(out)
    out = self.relu(out)
    out = self.l3(out) 
    # No activation, no softmax
    return out

In [95]:
model = NeuralNet(input_size, hidden_size, output_size).to(device)

In [96]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [100]:
# Optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

for epoch in range(num_epochs):
  for (words, labels) in train_loader:
    words = words.to(device)
    labels = labels.to(device)

    # Forward pass
    outputs = model(words)
    loss = criterion(outputs, labels)

    # Backward pass, optimizer step
    optimizer.zero_grad()
    loss.backward() # Backpropagation
    optimizer.step()

  # Every 100 steps
  if (epoch + 1) % 100 == 0:
    print(f"epoch {epoch+1}/{num_epochs}, loss={loss.item():.4f}")

print(f"Final loss = {loss.item():.4f}")

epoch 100/1000, loss=0.0000
epoch 200/1000, loss=0.0000
epoch 300/1000, loss=0.0000
epoch 400/1000, loss=0.0000
epoch 500/1000, loss=0.0000
epoch 600/1000, loss=0.0000
epoch 700/1000, loss=0.0000
epoch 800/1000, loss=0.0000
epoch 900/1000, loss=0.0000
epoch 1000/1000, loss=0.0000
Final loss = 0.0000


In [103]:
data = {
    "model_state": model.state_dict(),
    "input_size": input_size,
    "output_size": output_size,
    "hidden_size": hidden_size,
    "all_words": all_words,
    "tags": tags
}

FILE = "data.pth"
torch.save(data, FILE)

print(f"Training complete, file saved to {FILE}")

Training complete, file saved to data.pth


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Download pickled file
from google.colab import files
files.download('data.pth')

**Chatbot**

In [104]:
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data["all_words"]
tags = data["tags"]
model_state = data["model_state"]

In [105]:
model.load_state_dict(model_state)
model.eval()

NeuralNet(
  (l1): Linear(in_features=54, out_features=8, bias=True)
  (l2): Linear(in_features=8, out_features=8, bias=True)
  (l3): Linear(in_features=8, out_features=7, bias=True)
  (relu): ReLU()
)

In [113]:
import random
name = "Clarisse"
print("Hello there! Let's talk. \nP.S. Type 'quit' to exit.")

while True:
  sentence = input("You: ")
  if sentence == "quit":
    break

  # Tokenize sentence
  sentence = tokenize(sentence)

  # Create bag of words
  x = bag_of_words(sentence, all_words)
  x = x.reshape(1, x.shape[0]) 
  x = torch.from_numpy(x) # Convert to torch tensor

  output = model(x)

  _, predicted = torch.max(output, dim=1)
  tag = tags[predicted.item()] # tags[class label]

  probs = torch.softmax(output, dim=1)
  prob = probs[0][predicted.item()]

  if prob.item() > 0.75: # Only returns a response if the confidence is >0.75
    for intent in intents["intents"]:
      if tag == intent["tag"]:
        print(f"{name}: {random.choice(intent['responses'])}") # Gets a random response
  
  else:
    print(f"{name}: I do not understand...")
    

Hello there! Let's talk. 
P.S. Type 'quit' to exit.
You: hello
Clarisse: Hello, thanks for visiting
You: how are you
Clarisse: Hello, thanks for visiting
You: quit
